# Thrones2Vec

© Yuriy Guts, 2016

Using only the raw text of [A Song of Ice and Fire](https://en.wikipedia.org/wiki/A_Song_of_Ice_and_Fire), we'll derive and explore the semantic properties of its words.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function

In [2]:
import codecs
import glob
import logging
import multiprocessing
import os
import pprint
import re

In [3]:
import nltk
import gensim.models.word2vec as w2v
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

C:\Users\StarksForever\Anaconda2\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


**Set up logging**

In [5]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

**Download NLTK tokenizer models (only the first time)**

In [6]:
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\StarksForever\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\StarksForever\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Prepare Corpus

**Load books from files**

In [7]:
book_filenames = sorted(glob.glob("*.txt"))

In [8]:
print("Found books:")
book_filenames

Found books:


['got1.txt']

**Combine the books into one string**

In [9]:
corpus_raw = u""
for book_filename in book_filenames:
    print("Reading '{0}'...".format(book_filename))
    with codecs.open(book_filename, "r", "utf-8") as book_file:
        corpus_raw += book_file.read()
    print("Corpus is now {0} characters long".format(len(corpus_raw)))
    print()

Reading 'got1.txt'...
Corpus is now 1770659 characters long



**Split the corpus into sentences**

In [10]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [11]:
raw_sentences = tokenizer.tokenize(corpus_raw)

In [12]:
#convert into a list of words
#rtemove unnnecessary,, split into words, no hyphens
#list of words
def sentence_to_wordlist(raw):
    clean = re.sub("[^a-zA-Z]"," ", raw)
    words = clean.split()
    return words

In [13]:
#sentence where each word is tokenized
sentences = []
for raw_sentence in raw_sentences:
    if len(raw_sentence) > 0:
        sentences.append(sentence_to_wordlist(raw_sentence))

In [14]:
print(raw_sentences[5])
print(sentence_to_wordlist(raw_sentences[5]))

Heraldic crest by Virginia Norey.
[u'Heraldic', u'crest', u'by', u'Virginia', u'Norey']


In [15]:
token_count = sum([len(sentence) for sentence in sentences])
print("The book corpus contains {0:,} tokens".format(token_count))

The book corpus contains 331,572 tokens


## Train Word2Vec

In [16]:
#ONCE we have vectors
#step 3 - build model
#3 main tasks that vectors help with
#DISTANCE, SIMILARITY, RANKING

# Dimensionality of the resulting word vectors.
#more dimensions, more computationally expensive to train
#but also more accurate
#more dimensions = more generalized
num_features = 300
# Minimum word count threshold.
min_word_count = 3

# Number of threads to run in parallel.
#more workers, faster we train
num_workers = multiprocessing.cpu_count()

# Context window length.
context_size = 7

# Downsample setting for frequent words.
#0 - 1e-5 is good for this
downsampling = 1e-3

# Seed for the RNG, to make the results reproducible.
#random number generator
#deterministic, good for debugging
seed = 1

In [17]:
thrones2vec = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

In [18]:
thrones2vec.build_vocab(sentences)

2017-03-25 00:11:37,421 : INFO : collecting all words and their counts
2017-03-25 00:11:37,421 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-03-25 00:11:37,450 : INFO : PROGRESS: at sentence #10000, processed 140984 words, keeping 10280 word types
2017-03-25 00:11:37,480 : INFO : PROGRESS: at sentence #20000, processed 279730 words, keeping 13558 word types
2017-03-25 00:11:37,490 : INFO : collected 15178 word types from a corpus of 331572 raw words and 23532 sentences
2017-03-25 00:11:37,490 : INFO : Loading a fresh vocabulary
2017-03-25 00:11:37,581 : INFO : min_count=3 retains 7234 unique words (47% of original 15178, drops 7944)
2017-03-25 00:11:37,581 : INFO : min_count=3 leaves 321389 word corpus (96% of original 331572, drops 10183)
2017-03-25 00:11:37,601 : INFO : deleting the raw counts dictionary of 15178 items
2017-03-25 00:11:37,601 : INFO : sample=0.001 downsamples 50 most-common words
2017-03-25 00:11:37,601 : INFO : downsampling leaves e

In [19]:
print("Word2Vec vocabulary length:", len(thrones2vec.wv.vocab))

Word2Vec vocabulary length: 7234


**Start training, this might take a minute or two...**

In [20]:
thrones2vec.train(sentences)

2017-03-25 00:11:37,871 : INFO : training model with 4 workers on 7234 vocabulary and 300 features, using sg=1 hs=0 sample=0.001 negative=5 window=7
2017-03-25 00:11:37,871 : INFO : expecting 23532 sentences, matching count from corpus used for vocabulary survey
2017-03-25 00:11:38,904 : INFO : PROGRESS: at 20.09% examples, 241253 words/s, in_qsize 8, out_qsize 0
2017-03-25 00:11:39,908 : INFO : PROGRESS: at 42.11% examples, 256486 words/s, in_qsize 8, out_qsize 0
2017-03-25 00:11:40,934 : INFO : PROGRESS: at 64.34% examples, 260937 words/s, in_qsize 7, out_qsize 0
2017-03-25 00:11:41,944 : INFO : PROGRESS: at 86.06% examples, 262611 words/s, in_qsize 7, out_qsize 0
2017-03-25 00:11:42,513 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-03-25 00:11:42,532 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-03-25 00:11:42,582 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-03-25 00:11:42,592 : INFO : worker thread finis

1239728

**Save to file, can be useful later**

In [21]:
if not os.path.exists("trained"):
    os.makedirs("trained")

In [22]:
thrones2vec.save(os.path.join("trained", "thrones2vec.tsv"))

2017-03-25 00:11:42,713 : INFO : saving Word2Vec object under trained\thrones2vec.tsv, separately None
2017-03-25 00:11:42,723 : INFO : not storing attribute syn0norm
2017-03-25 00:11:42,723 : INFO : not storing attribute cum_table
2017-03-25 00:11:42,792 : INFO : saved trained\thrones2vec.tsv


## Explore the trained model.

In [23]:
thrones2vec = w2v.Word2Vec.load(os.path.join("trained", "thrones2vec.bin"))

2017-03-25 00:11:42,852 : INFO : loading Word2Vec object from trained\thrones2vec.bin
2017-03-25 00:11:42,891 : INFO : loading wv recursively from trained\thrones2vec.bin.wv.* with mmap=None
2017-03-25 00:11:42,894 : INFO : setting ignored attribute syn0norm to None
2017-03-25 00:11:42,894 : INFO : setting ignored attribute cum_table to None
2017-03-25 00:11:42,895 : INFO : loaded trained\thrones2vec.bin


### Compress the word vectors into 2D space and plot them

In [24]:
#my video - how to visualize a dataset easily
tsne = sklearn.manifold.TSNE(n_components=2, random_state=0)

In [ ]:
all_word_vectors_matrix = thrones2vec.wv.syn0

**Train t-SNE, this could take a minute or two...**

In [ ]:
all_word_vectors_matrix_2d = tsne.fit_transform(all_word_vectors_matrix)

**Plot the big picture**

In [ ]:
points = pd.DataFrame(
    [
        (word, coords[0], coords[1])
        for word, coords in [
            (word, all_word_vectors_matrix_2d[thrones2vec.wv.vocab[word].index])
            for word in thrones2vec.wv.vocab
        ]
    ],
    columns=["word", "x", "y"]
)

In [ ]:
points.head(10)

In [ ]:
sns.set_context("poster")

In [ ]:
points.plot.scatter("x", "y", s=10, figsize=(20, 12))

**Zoom in to some interesting places**

In [ ]:
def plot_region(x_bounds, y_bounds):
    slice = points[
        (x_bounds[0] <= points.x) &
        (points.x <= x_bounds[1]) & 
        (y_bounds[0] <= points.y) &
        (points.y <= y_bounds[1])
    ]
    
    ax = slice.plot.scatter("x", "y", s=35, figsize=(10, 8))
    for i, point in slice.iterrows():
        ax.text(point.x + 0.005, point.y + 0.005, point.word, fontsize=11)

**People related to Kingsguard ended up together**

In [ ]:
plot_region(x_bounds=(0, 1), y_bounds=(-4, -3))

**Food products are grouped nicely as well. Aerys (The Mad King) being close to "roasted" also looks sadly correct**

In [ ]:
plot_region(x_bounds=(0, 1), y_bounds=(-2, 0))

### Explore semantic similarities between book characters

**Words closest to the given word**

In [ ]:
thrones2vec.most_similar("Lily")

In [ ]:
thrones2vec.most_similar("Gringotts")

In [ ]:
thrones2vec.most_similar("Hogwarts")

**Linear relationships between word pairs**

In [ ]:
def nearest_similarity_cosmul(start1, end1, end2):
    similarities = thrones2vec.most_similar_cosmul(
        positive=[end2, start1],
        negative=[end1]
    )
    start2 = similarities[0][0]
    print("{start1} is related to {end1}, as {start2} is related to {end2}".format(**locals()))
    return start2

In [ ]:
nearest_similarity_cosmul("Harry", "Potter", "Ron")
nearest_similarity_cosmul("Ron", "Hermione", "Luna")
nearest_similarity_cosmul("Harry", "Voldemort","Hogwarts")
